In [55]:
import pandas as pd
import pandas as pd
import numpy as np
from keras.utils import np_utils
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.preprocessing import MinMaxScaler
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
import matplotlib.pyplot as plt
import itertools
from sklearn.metrics import roc_auc_score,confusion_matrix, accuracy_score, make_scorer, f1_score,precision_score,recall_score, plot_confusion_matrix
import seaborn as sns
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.tree import DecisionTreeClassifier
from keras.utils import np_utils

import time

In [56]:
def print_distribution(arr, print_nl=True):
    arr = np.unique(arr, return_counts=True)[1] / len(arr)
    for i in range(arr.shape[0]):
        print("Classe %d: %.2f%%" %(i, arr[i]*100))

    if print_nl:
        print("\n")

In [57]:
def taste(quality):
  if quality <= 5:
    return 0
  elif quality > 5 and quality <= 7:
    return 1
  else:
    return 2

In [58]:
time_begin = time.time()
data = pd.read_csv("wine_quality.csv")
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6497 entries, 0 to 6496
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed.acidity         6497 non-null   float64
 1   volatile.acidity      6497 non-null   float64
 2   citric.acid           6497 non-null   float64
 3   residual.sugar        6497 non-null   float64
 4   chlorides             6497 non-null   float64
 5   free.sulfur.dioxide   6497 non-null   float64
 6   total.sulfur.dioxide  6497 non-null   float64
 7   density               6497 non-null   float64
 8   pH                    6497 non-null   float64
 9   sulphates             6497 non-null   float64
 10  alcohol               6497 non-null   float64
 11  quality               6497 non-null   int64  
dtypes: float64(11), int64(1)
memory usage: 609.2 KB


In [59]:
new_data = data
new_data["taste"] = data["quality"].apply(taste)

In [60]:
new_data

,fixed.acidity,volatile.acidity,citric.acid,residual.sugar,chlorides,free.sulfur.dioxide,total.sulfur.dioxide,density,pH,sulphates,alcohol,quality,taste
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5,0
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.99680,3.20,0.68,9.8,5,0
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.99700,3.26,0.65,9.8,5,0
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.99800,3.16,0.58,9.8,6,1
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6492,6.2,0.21,0.29,1.6,0.039,24.0,92.0,0.99114,3.27,0.50,11.2,6,1
6493,6.6,0.32,0.36,8.0,0.047,57.0,168.0,0.99490,3.15,0.46,9.6,5,0
6494,6.5,0.24,0.19,1.2,0.041,30.0,111.0,0.99254,2.99,0.46,9.4,6,1
6495,5.5,0.29,0.30,1.1,0.022,20.0,110.0,0.98869,3.34,0.38,12.8,7,1


## Divisão de base de dados e separação de base

In [61]:
X = new_data.iloc[:, 0:11].values
y = new_data.iloc[:, 12].values

In [62]:
print("Proporções por classe no dataset em geral:")
print_distribution(y)

kf = StratifiedKFold(n_splits=10, shuffle=True, random_state=199)
fold = 0
for train, test in kf.split(X, y):
    print("Fold %d" %(fold))
    print_distribution(y[train], print_nl=(fold != 2))
    fold += 1

Proporções por classe no dataset em geral:
Classe 0: 36.69%
Classe 1: 60.26%
Classe 2: 3.05%


Fold 0
Classe 0: 36.69%
Classe 1: 60.27%
Classe 2: 3.04%


Fold 1
Classe 0: 36.69%
Classe 1: 60.27%
Classe 2: 3.04%


Fold 2
Classe 0: 36.69%
Classe 1: 60.27%
Classe 2: 3.04%
Fold 3
Classe 0: 36.69%
Classe 1: 60.27%
Classe 2: 3.04%


Fold 4
Classe 0: 36.70%
Classe 1: 60.25%
Classe 2: 3.04%


Fold 5
Classe 0: 36.70%
Classe 1: 60.25%
Classe 2: 3.04%


Fold 6
Classe 0: 36.70%
Classe 1: 60.25%
Classe 2: 3.04%


Fold 7
Classe 0: 36.70%
Classe 1: 60.24%
Classe 2: 3.06%


Fold 8
Classe 0: 36.70%
Classe 1: 60.24%
Classe 2: 3.06%


Fold 9
Classe 0: 36.70%
Classe 1: 60.26%
Classe 2: 3.04%




In [63]:
kf_accuracy = list()
kf_conf_mat = list()
kf_precision = list()
kf_f1 = list()
kf_recall = list()
kf_roc = list()

## Laço para execução do gridsearch em cada fold 

In [64]:
for train_ix, test_ix in kf.split(X,y):
    X_train, X_test = X[train_ix, :], X[test_ix, :]
    y_train, y_test = y[train_ix], y[test_ix]
    model = DecisionTreeClassifier(random_state=199)
    # parâmetros para teste do gridsearch
    parameters = dict()
    parameters["max_leaf_nodes"] = list(range(2, 20))
    parameters["min_samples_split"] = [2,4,6]
    parameters["max_depth"] = [4,6,8]
    parameters["criterion"] = ["gini","entropy"]
    search = GridSearchCV(model, parameters, scoring="accuracy", cv=10, refit=True)
    # executar e salvar melhores parâmetros do grisearch
    result = search.fit(X_train, y_train)
    best_model = result.best_estimator_
    # teste dos melhores parâmetros
    predicted_y = best_model.predict(X_test)
    #transformação de itens para cálculo do ROC
    y_test_dummy = np_utils.to_categorical(y_test)
    y_test_dummy
    predicted_y_dummy = np_utils.to_categorical(predicted_y)
    predicted_y_dummy
    # métricas do modelo
    acc = accuracy_score(y_test, predicted_y)
    conf_mat = confusion_matrix(y_test, predicted_y)
    precision = precision_score(y_test, predicted_y, average = "macro")
    f1 = f1_score(y_test, predicted_y, average = "macro")
    recall = recall_score(y_test, predicted_y, average = "macro")
    roc = roc_auc_score(y_test_dummy,predicted_y_dummy,multi_class="ovo",average = "macro")
    #armazenamento de métricas
    kf_accuracy.append(acc)
    kf_conf_mat.append(conf_mat)
    kf_precision.append(precision)
    kf_f1.append(f1)
    kf_recall.append(recall)
    kf_roc.append(roc)
    #métricas
    print(">acc=%.3f,>prec=%.3f,>f1=%.3f,>rec=%.3f,>roc=%.3f, est=%.3f, cfg=%s" % (acc,precision,f1,recall,roc, result.best_score_, result.best_params_))
    print(conf_mat)

C:\Users\cfpc2\anaconda3\envs\rv\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


>acc=0.715,>prec=0.469,>f1=0.472,>rec=0.476,>roc=0.705, est=0.687, cfg={'criterion': 'gini', 'max_depth': 4, 'max_leaf_nodes': 6, 'min_samples_split': 2}
[[147  92   0]
 [ 73 318   0]
 [  0  20   0]]


C:\Users\cfpc2\anaconda3\envs\rv\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


>acc=0.685,>prec=0.448,>f1=0.444,>rec=0.447,>roc=0.662, est=0.694, cfg={'criterion': 'gini', 'max_depth': 4, 'max_leaf_nodes': 6, 'min_samples_split': 2}
[[124 115   0]
 [ 70 321   0]
 [  0  20   0]]


C:\Users\cfpc2\anaconda3\envs\rv\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


...

C:\Users\cfpc2\anaconda3\envs\rv\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


>acc=0.675,>prec=0.441,>f1=0.428,>rec=0.431,>roc=0.640, est=0.689, cfg={'criterion': 'gini', 'max_depth': 4, 'max_leaf_nodes': 6, 'min_samples_split': 2}
[[106 132   0]
 [ 60 332   0]
 [  1  18   0]]


C:\Users\cfpc2\anaconda3\envs\rv\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


>acc=0.681,>prec=0.443,>f1=0.447,>rec=0.452,>roc=0.670, est=0.693, cfg={'criterion': 'gini', 'max_depth': 4, 'max_leaf_nodes': 6, 'min_samples_split': 2}
[[139  99   0]
 [ 89 303   0]
 [  0  19   0]]
>acc=0.710,>prec=0.465,>f1=0.468,>rec=0.472,>roc=0.700, est=0.693, cfg={'criterion': 'gini', 'max_depth': 4, 'max_leaf_nodes': 6, 'min_samples_split': 2}
[[145  93   0]
 [ 75 316   0]
 [  1  19   0]]


C:\Users\cfpc2\anaconda3\envs\rv\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [65]:
print("Accuracy: %.3f (%.3f)" % (np.mean(kf_accuracy), np.std(kf_accuracy)))
print("Precision: %.3f (%.3f)" % (np.mean(kf_precision), np.std(kf_precision)))
print("F1-score: %.3f (%.3f)" % (np.mean(kf_f1), np.std(kf_f1)))
print("Recall: %.3f (%.3f)" % (np.mean(kf_recall), np.std(kf_recall)))
print("ROC: %.3f (%.3f)" % (np.mean(kf_roc), np.std(kf_roc)))

Accuracy: 0.701 (0.015)
Precision: 0.460 (0.012)
F1-score: 0.457 (0.016)
Recall: 0.461 (0.016)
ROC: 0.684 (0.024)


In [66]:
kf_conf_mat

[array([[147,  92,   0],
        [ 73, 318,   0],
        [  0,  20,   0]], dtype=int64),
 array([[124, 115,   0],
        [ 70, 321,   0],
        [  0,  20,   0]], dtype=int64),
 array([[122, 117,   0],
        [ 50, 341,   0],
        [  0,  20,   0]], dtype=int64),
 array([[108, 131,   0],
        [ 47, 344,   0],
        [  0,  20,   0]], dtype=int64),
 array([[143,  95,   0],
        [ 74, 318,   0],
        [  0,  20,   0]], dtype=int64),
 array([[156,  82,   0],
        [ 88, 304,   0],
        [  1,  19,   0]], dtype=int64),
 array([[153,  85,   0],
        [ 75, 317,   0],
        [  1,  19,   0]], dtype=int64),
 array([[106, 132,   0],
        [ 60, 332,   0],
        [  1,  18,   0]], dtype=int64),
 array([[139,  99,   0],
        [ 89, 303,   0],
        [  0,  19,   0]], dtype=int64),
 array([[145,  93,   0],
        [ 75, 316,   0],
        [  1,  19,   0]], dtype=int64)]

In [67]:
np.mean(kf_conf_mat,axis=0)
# teste

array([[134.3, 104.1,   0. ],
       [ 70.1, 321.4,   0. ],
       [  0.4,  19.4,   0. ]])

In [68]:
time_end = time.time()
print((time_end - time_begin)/60)

32.898126022020975
